## Cleaning all messages from both users and contacts

### Method

- Read in the csv file
- extract just the emails to play with
- make a cleaning function
- check what the cleaning function is doing

In [60]:
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import sys

sys.path.insert(0, '../mining')
import connector
import query
from vader_sentiment_analysis import SentimentAnalyser
import os
from bs4 import BeautifulSoup
import re

In [61]:
df = pd.read_csv("../data/SELECT_ALL_from_all_msgs_from_contacts_users.csv", 
                 header = 0)
df = df.drop(['Unnamed: 0'],axis=1)

2016-08-15 12:56:45,233 | ERROR   | In #5 <-- ('127.0.0.1', 56328) to ('localhost', 5432) failed: SSHException('SSH session not active',)
Traceback (most recent call last):
  File "/home/sophie/anaconda3/envs/py27/lib/python2.7/SocketServer.py", line 596, in process_request_thread
    self.finish_request(request, client_address)
  File "/home/sophie/anaconda3/envs/py27/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/sophie/anaconda3/envs/py27/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/home/sophie/anaconda3/envs/py27/lib/python2.7/site-packages/sshtunnel.py", line 296, in handle
    raise HandlerSSHTunnelForwarderError(msg)
HandlerSSHTunnelForwarderError: In #5 <-- ('127.0.0.1', 56328) to ('localhost', 5432) failed: SSHException('SSH session not active',)


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 56328)
----------------------------------------


In [4]:
list(df)

['user_id', 'contact_id', 'relationship', 'channel', 'sent_at', 'message']

In [42]:
df_emails = df[df['channel'] == 'gmail']

In [43]:
emails = df_emails['message'].tolist()

In [51]:
df_emails['user_id'].unique()[0:10]

array([  10.,   35.,   47.,   59.,   81.,   96.,   97.,  114.,  116.,  144.])

In [62]:
df_emails[['user_id','contact_id','relationship','message']][df_emails['user_id'] == 12284]

,user_id,contact_id,relationship,message
2823142,12284.0,33098.0,Friend,Shit sorry completely forgot about this will s...
2823152,12284.0,33098.0,Friend,hmm let me know if its not there in the next f...
2823153,12284.0,33098.0,Friend,Enjoy you got anything planned
2823166,12284.0,33098.0,Friend,Hi.\n\nI remember you mentioned how much you l...
2823167,12284.0,33098.0,Friend,Definitely Zabb Elee. Spicy food is my one tru...
2823168,12284.0,33098.0,Friend,I was hoping you’d say that.\n\nSee you then!\...
2823189,12284.0,33098.0,Friend,Here’s the link. I expect a full report back.\...
2823193,12284.0,33098.0,Friend,I’m 2 songs in and I love it already :)\n
2823194,12284.0,33098.0,Friend,Come over tonight and listen to the rest?\n\n-...
2823204,12284.0,33098.0,Friend,Sent from my iPhone\n\nBegin forwarded message...


In [24]:
print df[['channel','message']][0:10]
print len(df)

  channel                                            message
0   gmail  hey aldo,\nit was nice to meet you the other d...
1   gmail  That sounds great! My only complication is a g...
2   gmail  Alright. It looks like I'll be going to this T...
3   gmail  Hey Aldo! Sorry for not getting back to you.. ...
4   gmail  Hey Aldo!\nSorry for the slow reply; I'd defin...
5   gmail  Hey Aldo! Thanks so much for thinking of me an...
6   gmail  Hey Aldo!\nNo problem re: disappearing. I've b...
7   gmail  you could've just e-mailed w/ an emoticon\n;)\...
8   gmail  hey aldo,\nhere is the link for the movie i wa...
9   gmail  feeling *mostly* alright :)\ndefinitely didn't...
2867448


In [35]:
sql_query = """SELECT DISTINCT contacts.user_id AS user_id,
feed_items.from_id AS contact_id, contact_types.name AS relationship, channels.name
AS channel, feed_items.send_at AS sent_at, feed_items.body AS message
FROM feed_items
JOIN channels
ON feed_items.channel_id = channels.id
JOIN contacts
ON feed_items.from_id = contacts.id
JOIN contact_types
ON contacts.contact_type_id = contact_types.id
JOIN users
ON contacts.user_id = users.id
WHERE contacts.is_fake = false
UNION
SELECT DISTINCT contacts.user_id AS user_id,
feed_items.from_id AS contact_id, contact_types.name AS relationship, 
channels.name
AS channel, feed_items.send_at AS sent_at, feed_items.body AS message
FROM feed_items
JOIN channels ON feed_items.channel_id = channels.id
JOIN feeds ON feeds.id = feed_items.feed_id
JOIN contacts ON contacts.user_id = feeds.contact_id
JOIN contact_types ON contacts.contact_type_id = contact_types.id
JOIN users ON contacts.user_id = users.id
WHERE contacts.is_fake = false
ORDER BY user_id, contact_id, sent_at"""

dbconnection = connector.ConnectDB()
conn = dbconnection.get_connection()
q = query.Query(conn, sql_query)
query_list = q.get_query_list()
query_df = q.get_query_dataframe()
#q.write_df_to_csv()
query_df.shape

(2763818, 6)

In [2]:
#df_emails2 = query_df[query_df['channel'] == 'gmail']
#df_emails2[['user_id','contact_id','relationship','message','sent_at']][df_emails2['user_id'] == 12284][0:5]

In [3]:
# find occurrences of forwarded message
# Print out lines that contain \"Jerk\" and \"Mean\" to look for reasons why I may have dumped them

print df_emails[df_emails.message.str.contains('Begin Forwarded Message')][0:5]

NameError: name 'df_emails' is not defined

In [ ]:
df = pd.read_csv("../data/SELECT_ALL_from_all_msgs_from_contacts_users.csv", 
                 header = 0)